# Load Results

In [8]:
from zubora_gabora.experiment.experiment_loader import ExperimentLoader
from scipy.stats import wilcoxon

# Load the experiment
ga_experiments = ExperimentLoader("experiments/ga/").experiment_data
aco_experiments = ExperimentLoader("experiments/aco/").experiment_data
num_experiments = len(ga_experiments["experiment_id"].unique())


# Compare using Wilcoxon signed-rank test

In [9]:
def is_better(a,b):
  return a < b

alpha = 0.05

for i in range(1,num_experiments+1):
  # Extract those where column experiment_id equals i
  ga_experiment = ga_experiments[ga_experiments["experiment_id"] == i]
  aco_experiment = aco_experiments[aco_experiments["experiment_id"] == i]

  ga_fitness = ga_experiment["fitness"]
  aco_fitness = aco_experiment["fitness"]

  print(f"Experiment {i}")
  print(f"GA Fitness: {ga_fitness.mean()} +/- {ga_fitness.std()}")
  print(f"ACO Fitness: {aco_fitness.mean()} +/- {aco_fitness.std()}")

  if is_better(ga_fitness, aco_fitness):
    print("GA is better than ACO in Quality")
  else:
    print("ACO is better than GA in Quality")

  res = wilcoxon(ga_fitness, aco_fitness)
  if res.p_value < alpha:
    print("The difference is statistically significant")
  else:
    print("The difference is not statistically significant")

  ga_evaluations = ga_experiment["n_evaluations"]
  aco_evaluations = aco_experiment["n_evaluations"]

  print(f"GA Evaluations: {ga_evaluations.mean()} +/- {ga_evaluations.std()}")
  print(f"ACO Evaluations: {aco_evaluations.mean()} +/- {aco_evaluations.std()}")

  if is_better(ga_evaluations, aco_evaluations):
    print("GA is better than ACO in Speed")
  else:
    print("ACO is better than GA in Speed")
  
  res = wilcoxon(ga_evaluations, aco_evaluations)
  if res.p_value < alpha:
    print("The difference is statistically significant")
  else:
    print("The difference is not statistically significant")
  print("------------------------------------------------")

Experiment 1
GA Fitness: 37.87096774193548 +/- 33.84645913088878
ACO Fitness: 29.51613003225807 +/- 0.5080005080007637


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().